## //---------EL FUTURO ES INCIERTO---------//

In [1]:
import pandas as pd
import numpy as np
import time

# FASE 0: Constantes y métodos

In [2]:
path='input/'
pd.set_option('display.max_columns', 500)

In [3]:
def agregarDummies(df,cols_cat):
    for categoria in cols_cat:
        df=df.merge(pd.get_dummies(df[categoria],prefix=categoria+'_'), 
                how='left',left_index=True,right_index=True)
    return df

# FASE 1: Carga de datos

In [4]:
df = pd.read_csv(path+"BaseHistoricoJobInicial.csv",sep=';',encoding='latin1',index_col=0,na_values=['-','NULL'])
print(df.shape)
df.head()

/usr/local/lib/python2.7/dist-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


(7178825, 15)


,Id_HistoricoJob,Id_Job,Id_Malla,Fecha_Carga_Scheduler,Fecha_Ejec_Inicio,Fecha_Ejec_Fin,Duracion,Promedio,Agente,Mxrc,Maxcmpc,Grupo,Force_Complete,CCF,Estado
0,559926,@DF2BYIX,02E2BXH9,2017-06-12 14:55:00,2017-06-12 22:04:07,2017-06-13 02:25:03,4H20,NaN,BFEFNFO,0.0,0,SCHE01,YES,NO,COMPLETE
1,559927,@DF2FCL2,02FBFCL4,2017-06-12 14:55:00,2017-06-13 04:34:18,2017-06-13 04:35:04,46,1M00,PCLBE98,0.0,0,SCHE01,NO,NO,COMPLETE
2,559928,@DF2FCL4,02FBFCL2,2017-06-12 14:55:00,2017-06-13 04:34:18,2017-06-13 04:35:45,1M27,1M00,PCLBE98,0.0,0,SCHE01,NO,NO,COMPLETE
3,559929,@DF2IACV,02FBFCL2,2017-06-12 14:55:00,2017-06-13 00:15:47,2017-06-13 00:15:49,2,NaN,BFEFNFO,0.0,0,SCHE01,NO,NO,COMPLETE
4,559930,@DF2WN92,02WNH992,2017-06-12 14:55:00,2017-06-12 20:26:28,2017-06-12 20:26:53,25,1M00,PQAVLE93,0.0,0,SCHE01,NO,NO,COMPLETE


In [5]:
df_backup=df.copy()

#### Limpieza de fechas

In [6]:
df['Fecha_Carga_Scheduler']=pd.to_datetime(df['Fecha_Carga_Scheduler'].str[:19])
df['Fecha_Ejec_Inicio']=pd.to_datetime(df['Fecha_Ejec_Inicio'].str[:19])
df['Fecha_Ejec_Fin']=pd.to_datetime(df['Fecha_Ejec_Fin'].str[:19])

In [7]:
df=df[df['Fecha_Ejec_Inicio'].notnull() & df['Fecha_Ejec_Fin'].notnull() & df['Fecha_Carga_Scheduler'].notnull()]
df.shape

(7168009, 15)

# FASE 2: Creación de variables

### Creación de variable duración

In [8]:
df['Duracion']=df['Fecha_Ejec_Fin']-df['Fecha_Ejec_Inicio']
df['Duracion_seg']=df['Duracion'].dt.seconds
df.loc[df['Duracion_seg']<1,'Duracion_seg']=1

### Creación de variable Horas entre Scheduler-Inicio

In [9]:
df['Minutos_Scheduler-Inicio']=(df['Fecha_Ejec_Inicio']-df['Fecha_Carga_Scheduler'])
df['Minutos_Scheduler-Inicio']=(df['Minutos_Scheduler-Inicio'].dt.seconds/60).map(int)
df.loc[df['Minutos_Scheduler-Inicio']<0,'Minutos_Scheduler-Inicio']=0

### Creación de variable ejecución en Mainframe

In [10]:
df['flg_Mainframe']=0
df.loc[df['Maxcmpc'].str[:4]=='NO S','flg_Mainframe']=1

### Creación de variable Grupo de error

In [11]:
#Segun lo leido en la descripcion de variables
df['Grupo_Error']='OTRO'
df.loc[df['Maxcmpc']=='0','Grupo_Error']=np.nan #Sin error
df.loc[df['Maxcmpc'].str[:3]=='CCF','Grupo_Error']='CCFAIL'
df.loc[df['Maxcmpc'].str[:3]=='JCL','Grupo_Error']='JCLERRO'
df.loc[df['Maxcmpc'].str[:1]=='S','Grupo_Error']='ABENDS' #Abnormal end sistema
df.loc[df['Maxcmpc'].str[:1]=='U','Grupo_Error']='ABENDU' #Abnormal end usuario
df.loc[df['Maxcmpc'].str[:3]=='SYS','Grupo_Error']='SYSERRO'

### Creación de variables dummies DiaSemana/Periodo

In [12]:
df['Dia_Semana_Scheduler']=df['Fecha_Carga_Scheduler'].dt.weekday
df['Dia_Semana_Inicio']=df['Fecha_Ejec_Inicio'].dt.weekday
df['Periodo_Dia_Scheduler']=pd.cut(df['Fecha_Carga_Scheduler'].dt.hour,[0,6,12,18,25],
                                     labels=['Madrugada','Manhana','Tarde','Noche'],right=False)
df['Periodo_Dia_Inicio']=pd.cut(df['Fecha_Ejec_Inicio'].dt.hour,[0,6,12,18,25],
                                     labels=['Madrugada','Manhana','Tarde','Noche'],right=False)

## Agregar dummies

In [13]:
cols_cat = ['Dia_Semana_Scheduler','Dia_Semana_Inicio','Periodo_Dia_Scheduler','Periodo_Dia_Inicio']
df = agregarDummies(df,cols_cat)

In [14]:
df.head()

,Id_HistoricoJob,Id_Job,Id_Malla,Fecha_Carga_Scheduler,Fecha_Ejec_Inicio,Fecha_Ejec_Fin,Duracion,Promedio,Agente,Mxrc,Maxcmpc,Grupo,Force_Complete,CCF,Estado,Duracion_seg,Minutos_Scheduler-Inicio,flg_Mainframe,Grupo_Error,Dia_Semana_Scheduler,Dia_Semana_Inicio,Periodo_Dia_Scheduler,Periodo_Dia_Inicio,Dia_Semana_Scheduler__0,Dia_Semana_Scheduler__1,Dia_Semana_Scheduler__2,Dia_Semana_Scheduler__3,Dia_Semana_Scheduler__4,Dia_Semana_Scheduler__5,Dia_Semana_Scheduler__6,Dia_Semana_Inicio__0,Dia_Semana_Inicio__1,Dia_Semana_Inicio__2,Dia_Semana_Inicio__3,Dia_Semana_Inicio__4,Dia_Semana_Inicio__5,Dia_Semana_Inicio__6,Periodo_Dia_Scheduler__Madrugada,Periodo_Dia_Scheduler__Manhana,Periodo_Dia_Scheduler__Tarde,Periodo_Dia_Scheduler__Noche,Periodo_Dia_Inicio__Madrugada,Periodo_Dia_Inicio__Manhana,Periodo_Dia_Inicio__Tarde,Periodo_Dia_Inicio__Noche
0,559926,@DF2BYIX,02E2BXH9,2017-06-12 14:55:00,2017-06-12 22:04:07,2017-06-13 02:25:03,04:20:56,NaN,BFEFNFO,0.0,0,SCHE01,YES,NO,COMPLETE,15656,429,0,NaN,0,0,Tarde,Noche,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1
1,559927,@DF2FCL2,02FBFCL4,2017-06-12 14:55:00,2017-06-13 04:34:18,2017-06-13 04:35:04,00:00:46,1M00,PCLBE98,0.0,0,SCHE01,NO,NO,COMPLETE,46,819,0,NaN,0,1,Tarde,Madrugada,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
2,559928,@DF2FCL4,02FBFCL2,2017-06-12 14:55:00,2017-06-13 04:34:18,2017-06-13 04:35:45,00:01:27,1M00,PCLBE98,0.0,0,SCHE01,NO,NO,COMPLETE,87,819,0,NaN,0,1,Tarde,Madrugada,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
3,559929,@DF2IACV,02FBFCL2,2017-06-12 14:55:00,2017-06-13 00:15:47,2017-06-13 00:15:49,00:00:02,NaN,BFEFNFO,0.0,0,SCHE01,NO,NO,COMPLETE,2,560,0,NaN,0,1,Tarde,Madrugada,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0
4,559930,@DF2WN92,02WNH992,2017-06-12 14:55:00,2017-06-12 20:26:28,2017-06-12 20:26:53,00:00:25,1M00,PQAVLE93,0.0,0,SCHE01,NO,NO,COMPLETE,25,331,0,NaN,0,0,Tarde,Noche,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,1


### Copia del mismo exitoso previo

In [15]:
df_no_error=df[df['Mxrc']==0].sort_values(['Id_Malla','Id_Job','Fecha_Ejec_Inicio'])
previoDF=pd.DataFrame(columns=['Id_HistoricoJob','Id_HistoricoJob_Previo', 'Frecuencia', 'Duracion_Acumulada'])

start_time = time.time()

tmp=np.chararray((df_no_error.shape[0],4), itemsize=100)

primero=True
i=0

for index,row in df_no_error.iterrows():
    inrow=np.chararray(4, itemsize=100)
    inrow[0] =str(row['Id_HistoricoJob'])
    inrow[1] ='nan'
    inrow[2] ='nan'
    inrow[3] = 'nan'
    if primero: 
        primero=False
        tmp[i]=inrow
        i+=1
        prevrow=row
        continue
    
    condition=(prevrow['Id_Job']==row['Id_Job']) & (prevrow['Id_Malla']==row['Id_Malla'])
    
    if condition:
        inrow[1] = str(prevrow['Id_HistoricoJob'])
        if(tmp[i-1][2] == 'nan'):
            inrow[2] = str(1)
            inrow[3] = str(prevrow['Duracion_seg'])
        else:
            inrow[2] = str(int(tmp[i-1][2]) + 1)
            inrow[3] = str(prevrow['Duracion_seg'] + int(tmp[i-1][3])) #Na + wa dsir
        
    tmp[i]=inrow
    i+=1
    prevrow=row

previoDF=pd.DataFrame(tmp,columns=['Id_HistoricoJob','Id_HistoricoJob_Previo', 'Frecuencia', 'Duracion_Acumulada'])
previoDF=previoDF.replace('nan',np.nan)

previoDF=previoDF[previoDF['Id_HistoricoJob_Previo'].notnull()]
previoDF['Id_HistoricoJob']=previoDF['Id_HistoricoJob'].map(int)
previoDF['Id_HistoricoJob_Previo']=previoDF['Id_HistoricoJob_Previo'].map(int)
previoDF['Frecuencia'] = previoDF['Frecuencia'].map(int)
previoDF['Duracion_Acumulada'] = previoDF['Duracion_Acumulada'].map(int)

previoDF=previoDF.merge(df, how='left',suffixes=('','_2'),left_on='Id_HistoricoJob_Previo',right_on='Id_HistoricoJob')
previoDF=previoDF.drop('Id_HistoricoJob_2',axis=1)
df=df.merge(previoDF,suffixes=('','_Previo'), how='left',left_on='Id_HistoricoJob',right_on='Id_HistoricoJob')

df['Promedio'] = df['Duracion_Acumulada'] / df['Frecuencia']

print("--- %s seconds ---" % (time.time() - start_time))  
#1000 reg -> 0 seg
#10000 reg -> 2 seg
#100000 reg -> 15 seg
#1000000 reg -> 170 seg

--- 1089.71258807 seconds ---


In [16]:
df['Intervalo_Ultima_Ejec'] = np.nan
df.loc[df['Fecha_Ejec_Fin_Previo'].notnull(),'Intervalo_Ultima_Ejec'] = (df['Fecha_Ejec_Inicio'] - df['Fecha_Ejec_Fin_Previo']).dt.seconds/60

In [17]:
df.corr()

,Id_HistoricoJob,Promedio,Mxrc,Duracion_seg,Minutos_Scheduler-Inicio,flg_Mainframe,Dia_Semana_Scheduler,Dia_Semana_Inicio,Dia_Semana_Scheduler__0,Dia_Semana_Scheduler__1,Dia_Semana_Scheduler__2,Dia_Semana_Scheduler__3,Dia_Semana_Scheduler__4,Dia_Semana_Scheduler__5,Dia_Semana_Scheduler__6,Dia_Semana_Inicio__0,Dia_Semana_Inicio__1,Dia_Semana_Inicio__2,Dia_Semana_Inicio__3,Dia_Semana_Inicio__4,Dia_Semana_Inicio__5,Dia_Semana_Inicio__6,Periodo_Dia_Scheduler__Madrugada,Periodo_Dia_Scheduler__Manhana,Periodo_Dia_Scheduler__Tarde,Periodo_Dia_Scheduler__Noche,Periodo_Dia_Inicio__Madrugada,Periodo_Dia_Inicio__Manhana,Periodo_Dia_Inicio__Tarde,Periodo_Dia_Inicio__Noche,Id_HistoricoJob_Previo,Frecuencia,Duracion_Acumulada,Mxrc_Previo,Duracion_seg_Previo,Minutos_Scheduler-Inicio_Previo,flg_Mainframe_Previo,Dia_Semana_Scheduler_Previo,Dia_Semana_Inicio_Previo,Dia_Semana_Scheduler__0_Previo,Dia_Semana_Scheduler__1_Previo,Dia_Semana_Scheduler__2_Previo,Dia_Semana_Scheduler__3_Previo,Dia_Semana_Scheduler__4_Previo,Dia_Semana_Scheduler__5_Previo,Dia_Semana_Scheduler__6_Previo,Dia_Semana_Inicio__0_Previo,Dia_Semana_Inicio__1_Previo,Dia_Semana_Inicio__2_Previo,Dia_Semana_Inicio__3_Previo,Dia_Semana_Inicio__4_Previo,Dia_Semana_Inicio__5_Previo,Dia_Semana_Inicio__6_Previo,Periodo_Dia_Scheduler__Madrugada_Previo,Periodo_Dia_Scheduler__Manhana_Previo,Periodo_Dia_Scheduler__Tarde_Previo,Periodo_Dia_Scheduler__Noche_Previo,Periodo_Dia_Inicio__Madrugada_Previo,Periodo_Dia_Inicio__Manhana_Previo,Periodo_Dia_Inicio__Tarde_Previo,Periodo_Dia_Inicio__Noche_Previo,Intervalo_Ultima_Ejec
Id_HistoricoJob,1.000000,0.002399,0.003326,0.002810,-0.041783,NaN,0.011985,0.013056,-0.006479,-0.008805,0.005570,0.000289,-0.001692,0.006198,0.009908,-0.004103,-0.010649,-0.000908,0.002482,0.003678,0.002046,0.010762,-0.019255,-0.002421,0.017638,-0.008277,-0.007950,0.007231,0.008316,-0.004326,0.999171,0.197837,0.094258,NaN,0.003112,-0.041484,NaN,0.008146,0.011216,-0.005085,-0.007900,0.006989,0.002691,-0.004129,0.003380,0.007765,-0.005082,-0.009387,0.000320,0.004084,0.003346,0.000982,0.008012,-0.020703,-0.003928,0.020329,-0.009685,-0.007818,0.006496,0.008214,-0.003818,0.015493
Promedio,0.002399,1.000000,NaN,0.984974,-0.120978,NaN,-0.027497,-0.038034,0.008031,0.008862,0.007488,0.008387,0.009061,-0.038574,-0.014049,0.018797,0.008243,0.007048,0.009009,0.006823,-0.035410,-0.026412,-0.028932,-0.036954,0.061268,-0.032307,-0.066741,-0.044031,0.231455,-0.083958,0.002015,-0.042328,0.802483,NaN,0.985908,-0.121160,NaN,-0.027872,-0.038428,0.008137,0.008856,0.008028,0.008413,0.008243,-0.038452,-0.014257,0.018886,0.008294,0.007920,0.008685,0.005943,-0.035366,-0.026347,-0.028936,-0.036947,0.061269,-0.032313,-0.066759,-0.044022,0.231485,-0.083948,-0.056849
Mxrc,0.003326,NaN,1.000000,-0.001599,0.012097,NaN,0.003208,0.002195,0.000486,-0.001903,-0.001450,-0.000508,-0.002946,0.005047,0.003304,0.000632,-0.000502,-0.002339,0.000425,-0.002308,0.001215,0.004445,-0.009073,-0.012865,0.019178,-0.008675,0.007476,-0.000103,0.002737,-0.009295,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Duracion_seg,0.002810,0.984974,-0.001599,1.000000,-0.123217,NaN,-0.030670,-0.036127,0.009015,0.009082,0.008773,0.009247,0.009478,-0.038943,-0.020042,0.015180,0.008378,0.008740,0.009391,0.007843,-0.035147,-0.026532,-0.028295,-0.036349,0.060000,-0.031586,-0.064640,-0.042782,0.226723,-0.083319,0.002622,-0.042011,0.790752,NaN,0.972418,-0.118690,NaN,-0.028815,-0.039941,0.008336,0.009741,0.009000,0.008924,0.004844,-0.038251,-0.013169,0.019455,0.008947,0.008967,0.009738,0.002746,-0.035476,-0.026351,-0.028495,-0.036688,0.060602,-0.031919,-0.065120,-0.043062,0.227816,-0.083377,-0.057015
Minutos_Scheduler-Inicio,-0.041783,-0.120978,0.012097,-0.123217,1.000000,NaN,0.002347,0.035798,0.004079,-0.002992,-0.008288,0.000200,-0.015213,0.048003,-0.035367,-0.042721,0.004418,-0.006329,-0.006695,-0.000627,0.103728,-0.056735,-0.018872,-0.200941,0.189215,-0.069026,

In [18]:
df.to_csv(path + "recien_horneado.csv",sep=';',encoding='latin1', index = False)